#Cбор данных

Рассмотрим сайт  http://fontanka.ru. На данном сайте нас интересует архив <a href =  'http://www.fontanka.ru/fontanka/arc/2015/all.html'>новостей</a> за 2015 год. Получим ссылки на каждый день 2015 года и сохраним в отдельный файл.

In [ ]:
# необходимые пакеты 
import lxml.html
import requests
import codecs
import os
import shutil
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
#получим страницу с архивом за январь
path = "http://www.fontanka.ru/fontanka/arc/2015/all.html"
response = requests.get(path)
if response.status_code == 200:
    html = response.content
    tree = lxml.html.document_fromstring(html)
    
    links = tree.xpath("""//div[@class='calendar-inner']/table[@class='blank_year']//tr/td[@class='blank_monthcell']
        /table/tbody/tr/td[@class='dom']/a/@href""")
    with open('fontanka-year2015-links', 'w') as ouf:
        for link in links:
            ouf.write("http://fontanka.ru%s\n"%(link))

Теперь имеются ссылки на архив новостей на каждый день 2015 года. Нужно перейти по каждой ссылке на соответствующую страницу. На этой странице скачать ссылки на новости. <br>
Каждая новость имеют категорию. Будем учитывать это.

In [ ]:
with codecs.open('news_category_links', 'w', encoding="utf8") as ouf:
    with open('fontanka-year2015-links', 'r') as inf:
        summ = 0
        for link in inf:
            print link.strip(),
            response = requests.get(link)
            if response.status_code == 200:
                html = response.content
                tree = lxml.html.document_fromstring(html)

                links = tree.xpath('''/html/body/div[@class="wrapper clearfix"]/div[@class="main-col 
                clearfix"]/div[@class="big_center ml30 fll"]/div[@class="calendar-widget"]
                /div[@class="calendar-inner transition open"]/div[@class="calendar-list"]/div
                [@class="calendar-item"]/div[@class="calendar-item-title"]/a''')
                print len(links),
                summ += len(links)
                print summ
                for l in links:
                    link = l.attrib['href']
                    if link[0] != "/":
                        continue
                    category = l.getnext().getchildren()[0].text_content()
                    
                    ouf.write("%s\t%s\n" % (category, link))          

Осталось скачать сами новости.

In [ ]:
main_path = u"http://www.fontanka.ru"
path_for_saving_news = u'./all_news/'
number_of_news_downloaded = 0

cnt = 0
threshold = 47121
with codecs.open('news_category_links', 'r', encoding='utf8') as inf:
    for line in inf:
        cnt += 1
        print cnt, 
        if cnt < threshold:
            continue
	
        category, link = line.split('\t')
        link = link.strip()
        try:
            response = requests.get(main_path + link)
        except:
            print "Something bad (no response) with %s"%link
            continue
        
        if category not in os.listdir(path_for_saving_news):
            os.mkdir(path_for_saving_news + category)
        
        path_for_news = path_for_saving_news[2:] + category + '/'
        
        if response.status_code == 200:
            
            try:
                number_of_news_downloaded += 1
                html = response.content
                tree = lxml.html.document_fromstring(html)
                txt = tree.xpath("""//div[@class='article']/div[@class='article_desc']
                /div[@class='article_fulltext']/p/text()""")
                filename = link.replace("/", "_")
                with codecs.open(path_for_news+filename, 'w', encoding="utf8") as ouf:
                    for t in txt:
                        ouf.write(t)
            except:
                print "Something bad with %s"%link
                    
        else:
            print "Something bad with %s"%link
        if (number_of_news_downloaded + threshold) % 10 == 1:
            print "%d \n"%(number_of_news_downloaded + threshold), 
            time.sleep(0.5)

Загрузилось около 47 тысяч статей. <br>
Но среди полученных файлов есть пустые файлы (статьи), так как иногда ссылки вели на сторонние сайты. Избавимся от этого мусора. Также удалим папки в которых менее 100 статей.

In [ ]:
path_with_news = './all_news/'
for d in os.listdir(path_with_news):
    path = path_with_news + d
    nFilesInDir = 0
    nDeletedFilesInDir = 0
    if os.path.isdir(path):
        print "Папка: '%s'"%d
        
        for f in os.listdir(path):
            filePath = path + '/' + f
            if os.path.getsize(filePath) < 100:
                nDeletedFilesInDir += 1
                os.remove(filePath)
            else:
                nFilesInDir += 1
    print "\tКоличество хороших файлов: %d, количество удаленных файлов: %d"%(nFilesInDir, nDeletedFilesInDir)
    if nFilesInDir < 100 :
        res = "Папка удалена"
        shutil.rmtree(path)
    else:
        res = "Папка оставлена"
    print "\tРезультат: %s"%res